In [1]:
var fs = require("fs")

In [2]:
var inFileName = "re2-exhaustive.txt"

In [3]:
var content = fs.readFileSync(inFileName, { encoding: "utf-8" })

In [4]:
var lines = content.split("\n")

In [5]:
var sections = []
var sectionStarted = false;
var sectionFinished = false;

var i = 0, l = lines.length;
var objs = [];

line = "" + lines[++i];

var title = "";

outer: while (true) {
    var line;
    header: do {
        while (true) {
            if (i >= l) break outer;
            if (line.startsWith("[ RUN")) break;
            if (line.startsWith("strings")) break header;
            line = "" + lines[++i];
        }
        title = line.substr(line.search("]") + 2);

        line = "" + lines[++i];
    } while(false);
    var obj = { title, strs: [], regexps: [] };
    console.log(obj.title)
    if (line !== "strings") throw "Unexpected line content";

    while (true) {
        if (i >= l) break outer;
        line = "" + lines[++i];
        if (line === "regexps") break;
        obj.strs.push(line);
    }
    
    var nstrs = obj.strs.length;
    console.log(nstrs)

    var regexp = undefined;

    while (true) {
        if (i >= l) break outer;
        line = "" + lines[++i];

        if (line.startsWith("[") || line.startsWith("strings")) {
            console.log("Break on " + line);
            obj.regexps.push(regexp);
            break;
        }
        
        if (line.startsWith('"')) {
            if (regexp !== undefined) obj.regexps.push(regexp);
            regexp = { regexp: line, results: [] };
            continue;
        }

        regexp.results.push(line);
    }
    
    objs.push(obj);
}


EgrepLiterals.Lowercase
40
Break on strings
EgrepLiterals.Lowercase
40
Break on strings
EgrepLiterals.Lowercase
40
Break on strings
EgrepLiterals.Lowercase
40
Break on [       OK ] EgrepLiterals.Lowercase (270 ms)
EgrepLiterals.MixedCase
21
Break on strings
EgrepLiterals.MixedCase
21
Break on strings
EgrepLiterals.MixedCase
21
Break on strings
EgrepLiterals.MixedCase
21
Break on [       OK ] EgrepLiterals.MixedCase (326 ms)
EgrepLiterals.FoldCase
43
Break on strings
EgrepLiterals.FoldCase
43
Break on strings
EgrepLiterals.FoldCase
43
Break on strings
EgrepLiterals.FoldCase
43
Break on [       OK ] EgrepLiterals.FoldCase (440 ms)
EgrepLiterals.UTF8
31
Break on strings
EgrepLiterals.UTF8
31
Break on strings
EgrepLiterals.UTF8
31
Break on strings
EgrepLiterals.UTF8
31
Break on [       OK ] EgrepLiterals.UTF8 (131 ms)
Repetition.Simple
127
Break on strings
Repetition.Simple
41
Break on [       OK ] Repetition.Simple (373 ms)
Repetition.Capturing
255
Break on strings
Repetition.Capturing
51

In [6]:
fs.writeFileSync("exhaustive.json", JSON.stringify(objs, null, 2), { encoding: "utf-8"})

In [7]:
var testCases = [];
for (var obj of objs) {
    var testCase = {
        name: obj.title,
        strs: obj.strs.map(s => { return s.substring(1, s.length - 1); }),
        regexs: obj.regexps.map(r => { var s = r.regexp; return s.substring(1, s.length - 1); })
    };
    testCases.push(testCase);
}

27

In [8]:
function pad(num, size){     return ('000000000' + num).substr(-size); } 

In [10]:
var current = 0;
for (var testCase of testCases) {
    var name = testCase.name.replace(".", "_");
    fs.writeFileSync(`test-cases/${pad(current, 3)}_${name}.json`, JSON.stringify(
        testCase, null, 2
    ), { encoding: "utf-8" })
    ++current;
}

27

In [13]:
function replaceAllMinusOnes(ary) {
    function isAllMinusOnes(a) {
        for (let i = 0; i < a.length; ++i) if (a[i] !== -1) return false;
        return true;
    }
    return isAllMinusOnes(ary) ? []: ary;
}

In [14]:
var testResults = [];
var number = /^[0-9]?[0-9]$/;
var divider = /[- ]/;
for (var obj of objs) {
    var testResult = { name: obj.title, ary: null };
    testResult.ary =
        obj.regexps.map(r => r.results.map(
            s => {
                let ary = s.split(";");
                return ary.map(s => {
                    let a = s.split(" ").map(ss =>
                        ss === "-" ? [-1, -1] : ss.split("-").map(x=> {
                            if (!x.match(number)) throw s;
                            return +x;
                        })
                    );
                    return replaceAllMinusOnes(Array.prototype.concat.apply([], a));
                });
            }
        ));
    testResults.push(testResult);
}

27

In [15]:
var current = 0;
for (var testResult of testResults) {
    var name = testResult.name.replace(".", "_");
    fs.writeFileSync(`test-results/${pad(current, 3)}_${name}.json`, JSON.stringify(
        testResult.ary, null, 2
    ), { encoding: "utf-8" })
    ++current;
}

27